## <center> <font color='purple'> Dynamic Ensemble Machine Learning Models (Mix Pool)</font></center> 
#### <center>Firuz Juraev (Sungkyunkwan Unniversity)</center>

### <font color='green'> Libraries 

#### <font color='blue'> Basic Libraries 

In [1]:
import pandas as pd
import numpy as np 
from numpy import mean
from numpy import std
import seaborn as sns 
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### <font color='blue'> Single ML Models Libraries 

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#### <font color='blue'> Static ML Models Libraries 

In [3]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#### <font color='blue'> DES Libraries 

In [4]:
from deslib.des import DESP
from deslib.des import KNORAE
from deslib.des import KNORAU
from deslib.des import METADES
from deslib.des import DESKNN # new 
from deslib.des import KNOP # new 

#### <font color='blue'> DCS Libraries 

In [5]:
from deslib.dcs import MCB

#### <font color='blue'> Processing Libraries 

In [6]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score, 
                             f1_score,
                             roc_auc_score, 
                             auc)
from sklearn.metrics import roc_curve, roc_auc_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#### <font color='blue'>Libraries for Critical Diagram 

In [7]:
from sklearn.model_selection import StratifiedKFold
from scipy.stats import wilcoxon, friedmanchisquare, rankdata
from Orange.evaluation import compute_CD, graph_ranks
from sklearn.metrics import mean_absolute_error

### <font color='green'> Load Dataset 

In [8]:
train_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_train_data_4.csv")
test_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_test_data_4.csv")

X_train = train_dataset_upsampled.drop(["DEAD"], axis=1) 
y_train = train_dataset_upsampled["DEAD"]

X_test = test_dataset_upsampled.drop(["DEAD"], axis=1) 
y_test = test_dataset_upsampled["DEAD"]

In [9]:
columns = X_train.columns.to_list()

### <font color='green'> Data Normalization

In [10]:
Min_max_scaler = MinMaxScaler().fit(X_train)

## Scaling 
X_train_mm_scaled = Min_max_scaler.transform(X_train)
X_test_mm_scaled = Min_max_scaler.transform(X_test)

## Numpy Array to DataFrame 
df_train_mm_scaled = pd.DataFrame(X_train_mm_scaled, columns = columns)
df_test_mm_scaled = pd.DataFrame(X_test_mm_scaled, columns = columns)

### <font color='green'> Feature Selection

In [11]:
tain_mm_scaled_df = df_train_mm_scaled[:]
tain_mm_scaled_df["DEAD"] = y_train 

test_mm_scaled_df = df_test_mm_scaled[:] 
test_mm_scaled_df["DEAD"] = y_test


importances = mutual_info_classif(df_train_mm_scaled, y_train)
feat_importance = pd.Series(importances, tain_mm_scaled_df.columns[0:len(tain_mm_scaled_df.columns)-1])
    
feat_importance = feat_importance.sort_values(ascending=False)
    
selected_features = feat_importance[:30]
selected_features_list_mm_scaled = selected_features.index.to_list()


tain_mm_scaled_df[selected_features_list_mm_scaled].head(2)

,temperature_mean,respRate_std,respRate_var,skinTemperature_var,skinTemperature_std,heartRate_std,heartRate_var,sao2_var,bpCuffMean_var,sao2_std,...,BIRTH_WEIGHT,bpCuffDiastolic_mean,bpCuffSystolic_mean,sao2_mean,temperature_var,temperature_std,glucometer_mean,bpCuffMean_mean,PLATELET,D10W_MEAN
0,0.775318,0.422245,0.178291,0.017977,0.134078,0.22911,0.057937,0.009568,0.053294,0.097817,...,0.264267,0.457143,0.602649,0.881455,0.015695,0.125281,0.312593,0.310696,0.281330,0.151138
1,0.709668,0.583964,0.341014,0.007512,0.086672,0.28258,0.086102,0.006551,0.020317,0.080938,...,0.437819,0.479330,0.644907,0.926056,0.003770,0.061399,0.272551,0.351928,0.392157,0.196784


In [12]:
X_train_mm = df_train_mm_scaled[selected_features_list_mm_scaled][:]
X_test_mm = df_test_mm_scaled[selected_features_list_mm_scaled][:]

### <font color='purple'> Hold-out Test (With Mix ML) - (+FS, +HO)

In [13]:
def get_individual_result(model, dsel_x, dsel_y, test_x, test_y): 
    model.fit(dsel_x, dsel_y)
    y_preds = model.predict(test_x) 
    yproba = model.predict_proba(test_x)[::,1] 
    
    acc = accuracy_score(test_y, y_preds)
    prec = precision_score(test_y, y_preds)
    rec = recall_score(test_y, y_preds)
    f1 = f1_score(test_y, y_preds)
    fpr, tpr, _ = roc_curve(test_y,  yproba) 
    auc = roc_auc_score(test_y, yproba)
    
    return {"acc": acc, "prec": prec, "rec": rec, "f1":f1, "fpr": fpr, "tpr":tpr, "auc": auc}
    

In [14]:
def hold_out_mix_ML(): 
    rng = np.random.RandomState(42) 
    X_train, X_dsel, y_train_en, y_dsel = train_test_split(X_train_mm, y_train, test_size=0.40, random_state=rng)
    
    model_svc = SVC(kernel='linear', C=0.007, gamma=0.2, degree=3, probability=True, class_weight='balanced')
#     model_dt = DecisionTreeClassifier(criterion='entropy', max_depth=3) # depth was 3  
    model_lr = LogisticRegression(penalty='l2', C=0.002)
    model_ml_perceptron = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    
    model_rf  = RandomForestClassifier(criterion='gini', n_estimators=100, max_depth=3)
    model_cat = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
#     model_lgb = LGBMClassifier(max_depth=1, n_estimators=150, objective="binary")
#     model_ada = AdaBoostClassifier(n_estimators=250, learning_rate=0.01)
    
    model_svc.fit(X_train, y_train_en)
#     model_dt.fit(X_train, y_train_en)
    model_lr.fit(X_train, y_train_en)
    model_ml_perceptron.fit(X_train, y_train_en)
    model_rf.fit(X_train, y_train_en) 
    model_cat.fit(X_train, y_train_en) 
#     model_lgb.fit(X_train, y_train_en)
#     model_ada.fit(X_train, y_train_en)
    
    classifiers_names = ["MLP", "Logistic Regression", "SVC", "Random Forest", 
                         "CatBoost"] # "Decision Tree",
    # "AdaBoost" , "LGBM"
    pool_classifiers = [model_ml_perceptron, 
                        model_lr, 
                        model_svc, 
                        model_rf, 
                        model_cat] 
    c_acc_list = [] 
    
    for cls in pool_classifiers:
        y_preds_c = cls.predict(X_test_mm) 
        c_acc_list.append(accuracy_score(y_test, y_preds_c))           
        
        
    classifiers_results =  {'name': classifiers_names, 
                            'accuracy': c_acc_list}
    
    clsDF = pd.DataFrame.from_dict(classifiers_results)
        
    # DES STARTS
    
    knorau = KNORAU(pool_classifiers)
    kne = KNORAE(pool_classifiers)  
    metades = METADES(pool_classifiers)
    desknn = DESKNN(pool_classifiers)
    mcb = MCB(pool_classifiers)
    desp = DESP(pool_classifiers)
    knop = KNOP(pool_classifiers)

    fire_knorau = KNORAU(pool_classifiers, DFP=True, k=9) 
    fire_kne = KNORAE(pool_classifiers, DFP=True, k=9)
    fire_metades = METADES(pool_classifiers, DFP=True, k=9) 
    fire_desknn = DESKNN(pool_classifiers, DFP=True, k=9)
    fire_mcb = MCB(pool_classifiers, DFP=True, k=7) # 7 was 96%
    fire_desp = DESP(pool_classifiers, DFP=True, k=9)
    fire_knop = KNOP(pool_classifiers, DFP=True, k=15)
    
    ensemble_classifiers = [fire_knorau, knorau, fire_kne, kne, fire_metades, metades, fire_desknn, desknn, 
                            fire_mcb, mcb, fire_desp, desp, fire_knop, knop]
    
    ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                      "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]
    
    acc_list = [] 
    precision_list = [] 
    recall_list = []
    f1_lists = [] 
    auc_list = [] 
    fpr_list = [] 
    tpr_list = []
    
    for e_cls in ensemble_classifiers:
        results_dct = get_individual_result(e_cls, X_dsel, y_dsel, X_test_mm, y_test)
        acc_list.append(results_dct["acc"])
        precision_list.append(results_dct["prec"])
        recall_list.append(results_dct["rec"]) 
        f1_lists.append(results_dct["f1"]) 
        auc_list.append(results_dct["auc"])
        fpr_list.append(results_dct["fpr"])
        tpr_list.append(results_dct["tpr"])

    
    results =  {'name': ensemble_names, 
                'accuracy': acc_list, 
                'precision': precision_list, 
                'recall': recall_list, 'f1': f1_lists, 
                'auc': auc_list, 
                'tpr': tpr_list, 'fpr': fpr_list}
    
    df = pd.DataFrame.from_dict(results)
    
    return df, clsDF

In [15]:
results_data = []
classifier_results_data = []
for i in range(0, 15):
    result, cls_results = hold_out_mix_ML()
    results_data.append(result)
    classifier_results_data.append(cls_results)
    

fireResultsDF = pd.concat(results_data)
classifiersResultsDF = pd.concat(classifier_results_data)

Iteration 1, loss = 0.61854149
Iteration 2, loss = 0.50743715
Iteration 3, loss = 0.42067253
Iteration 4, loss = 0.37008862
Iteration 5, loss = 0.34133787
Iteration 6, loss = 0.32007949
Iteration 7, loss = 0.30783163
Iteration 8, loss = 0.29334780
Iteration 9, loss = 0.28444789
Iteration 10, loss = 0.27674642
Iteration 11, loss = 0.27403241
Iteration 1, loss = 0.64591622
Iteration 2, loss = 0.51578962
Iteration 3, loss = 0.42516850
Iteration 4, loss = 0.37370160
Iteration 5, loss = 0.34555199
Iteration 6, loss = 0.32404893
Iteration 7, loss = 0.30872979
Iteration 8, loss = 0.29710812
Iteration 9, loss = 0.28803194
Iteration 10, loss = 0.27824820
Iteration 11, loss = 0.27196749
Iteration 1, loss = 0.67143961
Iteration 2, loss = 0.57004588
Iteration 3, loss = 0.47407059
Iteration 4, loss = 0.40706781
Iteration 5, loss = 0.36665106
Iteration 6, loss = 0.34651367
Iteration 7, loss = 0.32460170
Iteration 8, loss = 0.31165842
Iteration 9, loss = 0.30312343
Iteration 10, loss = 0.29297642
Ite

Iteration 1, loss = 0.62657714
Iteration 2, loss = 0.51203645
Iteration 3, loss = 0.42800893
Iteration 4, loss = 0.37917377
Iteration 5, loss = 0.35148355
Iteration 6, loss = 0.33249318
Iteration 7, loss = 0.31796081
Iteration 8, loss = 0.30350462
Iteration 9, loss = 0.29442701
Iteration 10, loss = 0.29137402
Iteration 11, loss = 0.28621395
Iteration 1, loss = 0.65513031
Iteration 2, loss = 0.55343618
Iteration 3, loss = 0.46621373
Iteration 4, loss = 0.40520886
Iteration 5, loss = 0.37024582
Iteration 6, loss = 0.34560638
Iteration 7, loss = 0.32866648
Iteration 8, loss = 0.31400924
Iteration 9, loss = 0.30191467
Iteration 10, loss = 0.29291854
Iteration 11, loss = 0.28531199
Iteration 1, loss = 0.64259633
Iteration 2, loss = 0.53874625
Iteration 3, loss = 0.45825316
Iteration 4, loss = 0.40128388
Iteration 5, loss = 0.36985886
Iteration 6, loss = 0.34494730
Iteration 7, loss = 0.32850829
Iteration 8, loss = 0.31646183
Iteration 9, loss = 0.30436166
Iteration 10, loss = 0.29459436
Ite

In [16]:
ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                  "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]

accuracy = [] 
accuracy_std =  [] 
precision = [] 
precision_std = [] 
recall = [] 
recall_std = [] 
f1_score = [] 
f1_std = []
auc_score = []
auc_std = []


for n in ensemble_names:
    accuracy.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.mean(), 3))
    accuracy_std.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.std(), 3))
    precision.append(round(fireResultsDF[fireResultsDF.name == n].precision.mean(), 3))
    precision_std.append(round(fireResultsDF[fireResultsDF.name == n].precision.std(), 3))
    recall.append(round(fireResultsDF[fireResultsDF.name == n].recall.mean(), 3))
    recall_std.append(round(fireResultsDF[fireResultsDF.name == n].recall.std(), 3))
    f1_score.append(round(fireResultsDF[fireResultsDF.name == n].f1.mean(), 3))
    f1_std.append(round(fireResultsDF[fireResultsDF.name == n].f1.std(), 3))
    auc_score.append(round(fireResultsDF[fireResultsDF.name == n].auc.mean(), 3))
    auc_std.append(round(fireResultsDF[fireResultsDF.name == n].auc.std(), 3))
    
final_results = {"method": ensemble_names, 
                     "accuracy": accuracy, 
                     "accuracy_std": accuracy_std,
                     "precision": precision, 
                     "precision_std": precision_std,
                     "recall": recall, 
                     "recall_std": recall_std,
                     "f1_score": f1_score, 
                     "f1_std": f1_std, 
                     "auc": auc_score, 
                     "auc_std": auc_std}

finalResultsDF = pd.DataFrame.from_dict(final_results)

In [17]:
finalResultsDF

,method,accuracy,accuracy_std,precision,precision_std,recall,recall_std,f1_score,f1_std,auc,auc_std
0,FIRE-KNORA-U,0.937,0.013,0.923,0.005,0.953,0.030,0.937,0.013,0.982,0.001
1,KNORA-U,0.936,0.012,0.923,0.006,0.953,0.030,0.937,0.013,0.983,0.001
2,FIRE-KNORA-E,0.943,0.014,0.935,0.005,0.953,0.030,0.944,0.015,0.985,0.001
3,KNORA-E,0.944,0.014,0.936,0.005,0.953,0.030,0.944,0.015,0.985,0.001
4,FIRE-METADES,0.955,0.017,0.935,0.007,0.977,0.034,0.955,0.018,0.987,0.001
5,METADES,0.954,0.016,0.935,0.007,0.977,0.034,0.955,0.017,0.987,0.001
6,FIRE-DESKNN,0.943,0.002,0.950,0.005,0.935,0.000,0.943,0.002,0.985,0.001
7,DESKNN,0.943,0.002,0.951,0.004,0.935,0.000,0.943,0.002,0.985,0.001
8,FIRE-MCB,0.952,0.003,0.938,0.005,0.970,0.008,0.953,0.003,0.984,0.001
9,MCB,0.952,0.003,0.936,0.005,0.970,0.005,0.953,0.003,0.984,0.001


In [22]:
# finalResultsDF.to_csv("Results/mix_pool_results/des_mix_pool_5_cls_results_3-2.csv", index=False)

In [19]:
classifiers_names = ["MLP", "Logistic Regression", "SVC", "Random Forest", "CatBoost"] 
# "AdaBoost", "LGBM"

accuracy_list = []
accuracy_std_list = []
for n in classifiers_names:
    accuracy_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.mean())
    accuracy_std_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.std())

final_cls_results = {"classifier": classifiers_names, 
                     "accuracy": accuracy_list, 
                     "accuracy_std": accuracy_std_list}

finalClassifierResultsDF = pd.DataFrame.from_dict(final_cls_results)

In [20]:
finalClassifierResultsDF

,classifier,accuracy,accuracy_std
0,MLP,0.938979,1.981459e-02
1,Logistic Regression,0.941748,4.596760e-16
2,SVC,0.932578,0.000000e+00
3,Random Forest,0.937972,2.420755e-03
4,CatBoost,0.940921,1.312966e-02


In [21]:
finalResultsDF.describe().T

,count,mean,std,min,25%,50%,75%,max
accuracy,14.0,0.945071,6.145221e-03,0.936,0.94150,0.9430,0.95100,0.955
accuracy_std,14.0,0.011643,6.570413e-03,0.002,0.00525,0.0130,0.01550,0.022
precision,14.0,0.935786,8.087774e-03,0.923,0.93275,0.9355,0.93800,0.951
precision_std,14.0,0.005500,1.160239e-03,0.004,0.00500,0.0050,0.00600,0.008
recall,14.0,0.956357,1.331631e-02,0.935,0.95300,0.9530,0.96750,0.977
recall_std,14.0,0.024714,1.487087e-02,0.000,0.01350,0.0300,0.03300,0.045
f1_score,14.0,0.945500,6.260376e-03,0.937,0.94225,0.9435,0.95175,0.955
f1_std,14.0,0.012143,6.959853e-03,0.002,0.00550,0.0130,0.01650,0.023
auc,14.0,0.984857,1.460092e-03,0.982,0.98400,0.9850,0.98500,0.987
auc_std,14.0,0.001000,4.500519e-19,0.001,0.00100,0.0010,0.00100,0.001
